# Training a BAFF and MikadoRR model

In [1]:
import json
from pathlib import Path

import numpy as np
from ase.io import Trajectory
from ase import units

from cgf.utils.training import get_optimal_coeffs
from numpy.polynomial.polynomial import polyfit

## 1. Reading the training-data

The trajectory contains 41 datapoints of the 2D polymer COF-5, of which the first one is the relaxed structure, the next 20 are isotropically strained structures from -1.0% to 1.0, and the last 20 are sheared structures from -4.0% to 4.0% shear strain.

Furthermore, we will need later on `r0` which is the distance between two core vertex positions and can be extracted directly from the unit cell for a honeycomb lattice.

In [2]:
traj = Trajectory(Path('traj_training.traj'))

r0 = traj[0].cell.cellpar()[0]/np.sqrt(3)  # this is the distance between two node positions


Let us extract the energies and structures from the trajectory:

In [3]:
structures = []
energies = []
for n, atoms in enumerate(traj):
    del atoms.constraints
    energies.append(atoms.get_potential_energy())
    structures.append(atoms)
    if 1.2*r0>=atoms.cell.cellpar()[2]:  # in case cell z-coordinate to small
        cellnew = atoms.cell
        cellnew[2][2] = 2*r0
        atoms.set_cell(cellnew)

energies = np.array(energies)

## 2. Training the BAFF model from EoS fitting of the 2D Bulk and Shear moduli

From this we can directly obtain the 2D bulk and shear modulus via equation of state fitting. This we will need for the parametrization of the BAFF model:

In [4]:
### calc 2d bulk and shear modulus for BAFF: see https://doi.org/10.1021/acs.jpcc.2c06268
areas = [atoms.get_volume()/atoms.cell.cellpar()[2] for atoms in structures]
## bulk
popt, data = polyfit(x=areas[:21], y=energies[:21], deg=3, full=True)
res_B = data[0][0]
# 2D Bulk modulus as Aopt*d^2E/dA^2 at Aopt in eV/Angstrom^2
B = 2*np.sqrt(popt[2]**2 - 3*popt[1]*popt[3]) * areas[0]
print("B2D [N/m]: ", B * units.m**2 / units.J)

B2D [N/m]:  25.929674561479185


In [5]:
## shear
strains = [((structures[0].cell[0][0] - atoms.cell[0][0])/structures[0].cell[0][0]) for atoms in structures]
popt, data = polyfit(strains[21:], energies[21:], 3, full=True)
res_mu = data[0][0]

# shear modulus (dE^2/ds^2)/4Aopt at smin in eV/Angstrom^2
mu = 2*np.sqrt(popt[2]**2 - 3*popt[1]*popt[3])/(4*areas[0])
print("mu2D [N/m]: ", mu * units.m**2 / units.J)

mu2D [N/m]:  3.6665643988022443


Calculating the bonding and angular force-constants of BAFF from the 2D bulk and shear moduli:

In [6]:
training_model = dict()
training_model['B'] = B
training_model['res_B'] = res_B
training_model['mu'] = mu
training_model['res_mu'] = res_mu

training_model['Kbond'] =  2*np.sqrt(3) * B / 2
training_model['Kangle'] = (1/(mu*np.sqrt(3)) - 1/(2*np.sqrt(3) * B))**(-1) /9 * r0**2
training_model['cosT0'] = -0.5
print('----- \n Training model BAFF: \n', training_model)

with open(Path('training_model_BAFF.json'), 'w') as fp:
    json.dump(training_model, fp)

----- 
 Training model BAFF: 
 {'B': 1.618403004066553, 'res_B': 6.753100718691878e-08, 'mu': 0.2288489515576287, 'res_mu': 1.0201762839358919e-05, 'Kbond': 2.80315623016537, 'Kangle': 15.019847417402843, 'cosT0': -0.5}


## 3. Training the MikadoRR model by automatically extracting necessary features

We are automatically extracting the necessary features for the model and then fitting via a ridge-regression procedure the model.

For the feature extraction, we need to define `id_groups`. This is a list of lists, in which each list is a series of ids corresponding to atoms in the structure from which the center of geometry of the core is obtained. For instance `id_groups = [[30, 40, 50, 25, 45, 35], [10, 60, 20, 65, 15, 55]]` means that we have two cores. The position of core 1 is defined by the center of geometry of atoms [30, 40, 50, 25, 45, 35] and position of core 2 by the center of geometry of atoms [10, 60, 20, 65, 15, 55].

Then, the function `get_optimal_coeffs` is a helper function which automatically extracts the distances between the cores and all the angles describing the elastic beams.

Within this function, different linkage lengths are being probed to see which one works the best to fit a model. The linkage length divides the 2D polymer into a core region and a linker region, where the linkage length is the distance between the core center and the onset of the linkage.

In [7]:
### calc training for MikadoRR

with open(Path('id_groups_cores.json'), 'r') as inp:
    id_groups = json.load(inp)['id_groups']

# get coeffs with all available structures
results = get_optimal_coeffs(r0, structures, energies, id_groups=id_groups, width=4)
training_model = results['opt_training_model']
if training_model==None:
    print('----- \n No optimal training model MikadoRR found...\n')
else:
    print('----- \n Optimal training model MikadoRR: \n', training_model)

    with open(Path('training_model_MikadoRR.json'), 'w') as fp:
        json.dump(training_model, fp)
with open(Path('all_training_results_MikadoRR.json'), 'w') as fp:
    json.dump(results, fp)

Linkage length:  0.1
Training successful with cross_val_score_mean: 0.9984012881900722
Linkage length:  0.4034883569074885
Training successful with cross_val_score_mean: 0.9987474289509974
Linkage length:  0.7069767138149771
Training successful with cross_val_score_mean: 0.9985149171843695
Linkage length:  1.0104650707224656
Training successful with cross_val_score_mean: 0.9987410711880648
Linkage length:  1.3139534276299543
Training successful with cross_val_score_mean: -1.8160868935738543
Linkage length:  1.617441784537443
Training successful with cross_val_score_mean: 0.9987667185516832
Linkage length:  1.9209301414449313
Training successful with cross_val_score_mean: 0.9980311611350304
Linkage length:  2.22441849835242
Training successful with cross_val_score_mean: 0.9985036796970289
Linkage length:  2.5279068552599084


/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=9.55364e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=9.46448e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.03305e-16): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.07361e-16): result may not be accurate.
  return linalg.solve

Training successful with cross_val_score_mean: -0.2578986271347411
Linkage length:  2.831395212167397
Training successful with cross_val_score_mean: 0.9984799412903717
Linkage length:  3.1348835690748857
Training successful with cross_val_score_mean: 0.9950176585652054
Linkage length:  3.438371925982374
Training successful with cross_val_score_mean: 0.9987647346530562
Linkage length:  3.7418602828898626
Training successful with cross_val_score_mean: 0.9986970861941481
Linkage length:  4.045348639797351
Training successful with cross_val_score_mean: 0.9988131877260178
Linkage length:  4.348836996704839
Training successful with cross_val_score_mean: 0.998778906654451
Linkage length:  4.652325353612328
Training successful with cross_val_score_mean: 0.9988023284501384
Linkage length:  4.955813710519816
Training successful with cross_val_score_mean: 0.9885319454229211
Linkage length:  5.2593020674273046
Training successful with cross_val_score_mean: 0.9988004952445699
Linkage length:  5.562

/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=8.06436e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=6.63707e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=6.56743e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.14707e-17): result may not be accurate.
  return linalg.solve

Training successful with cross_val_score_mean: -7.288203300665767
Linkage length:  6.169767138149771


...

/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=8.68507e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Training successful with cross_val_score_mean: -8056.014339163196
Linkage length:  8.597673993409678


/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.07587e-16): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=8.86642e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=8.78368e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=9.57031e-17): result may not be accurate.
  return linalg.solve

Training successful with cross_val_score_mean: 0.9984856884345056
Linkage length:  8.901162350317167
Training successful with cross_val_score_mean: 0.9984856884341634
----- 
 Optimal training model MikadoRR: 
 {'rr_coeff': [-49.85999701479752, 1.4003018666893938, 16.756783067995613, 16.756783068003514, 16.756782849533202, 30.551767656051386], 'rr_incpt': 443.8363156932706, 'rr_alpha': 1e-07, 'R2': 0.9999893002464733, 'MSE': 2.8917666867956276e-07, 'cross_val_score_mean': 0.9988232362963998, 'n_samples': 41, 'linkage_length': 5.562790424334794, 'training_ID': 18}


/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.07587e-16): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=8.86642e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=8.78368e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/dbodesheim/anaconda3/envs/cgf_piptest/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=9.57031e-17): result may not be accurate.
  return linalg.solve